<center><u><b><h1 style="color:Red;">Titanic : Machine Learning from Disaster</u></b></h1></center>

<b>Loading Data</b>

In [97]:
import pandas as pd
import numpy as np
df_train = pd.read_csv(r'C:\Kaggles\Titanic\Dataset\train.csv')
df_test = pd.read_csv(r'C:\Kaggles\Titanic\Dataset\test.csv')
test_ids = df_test["PassengerId"]

print(df_train.shape)
df_train.head(3)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


<b>Data Cleaning</b>

In [98]:
def clean(data):
    data = data.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    return data

df_train = clean(df_train)
df_test = clean(df_test)

<b> Data cleaned up. </b>

In [99]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
categorical_columns = ["Sex", "Embarked"]

for column in categorical_columns:
    df_train.loc[:, column] = le.fit_transform(df_train[column])
    df_test.loc[:, column] = le.fit_transform(df_test[column])

df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [100]:
from sklearn.model_selection import train_test_split

X = df_train.drop("Survived", axis="columns")
Y = df_train["Survived"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.12, random_state=42)

In [101]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(verbose = 0)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.8598130841121495

In [102]:
submission_preds = model.predict(df_test)

In [103]:
df = pd.DataFrame({"PassengerId": test_ids.values,
                   "Survived": submission_preds,
                  })
df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [104]:
df.to_csv(r"C:\Kaggles\Titanic\Submissions\Submission_3.csv", index=False)